# Introduction to NLP fundamentals in Tensorflow


NLP - deriving information out of natural language


In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-e26afeb3-09cf-8264-34a0-5242a80f94dd)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/krishnamenon215/Tensorflow/main/helper_functions.py

--2023-01-23 03:16:30--  https://raw.githubusercontent.com/krishnamenon215/Tensorflow/main/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2023-01-23 03:16:31 (91.6 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#Import series of helper functions for our notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

## Get text dataset

dataset used: Kaggles introduction to NLP dataset(text samples of tweets labelled as disaster or not)

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

--2023-01-23 03:16:35--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.31.128, 74.125.28.128, 108.177.13.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.31.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.009s  

2023-01-23 03:16:35 (63.5 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



In [5]:
#unzip data
unzip_data("nlp_getting_started.zip")

## Visualizing a text dataset

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df["text"][1]

'Forest fire near La Ronge Sask. Canada'

In [8]:
# shuffle training dataframe
train_df_shuffled=train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
#visualise test_data
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
#examples of each class
train_df_shuffled.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
#How many total samples
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Lets visualise soem random training examples
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
  _,text,target=row
  print(f"Target:{target}","(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:{text}")
  print("\n")


Target:1 (real disaster)
Text:The 46 returned refugees - what were they fleeing from &amp; how will the Vietnamese Govt treat them now they are returned? #Dutton #presser


Target:1 (real disaster)
Text:I know it's a question of interpretation but this is a sign of the apocalypse.  I called it https://t.co/my8q1uWIjn


Target:0 (not real disaster
Text:Lab today ready for these body bags. ??


Target:0 (not real disaster
Text:Uhhhhh demon hunters. But not the whole Burning Crusade v 2.0 thing.  https://t.co/oPtpS1lgKC


Target:0 (not real disaster
Text:Check out Vintage Longaberger Floral Fabric Shoulder Cross Body Bag Brown Leather Strap http://t.co/FB8snRg4HU @eBay




### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
#split training data into training and validation test
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [15]:
# check the lengths 
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [16]:
#Check first 10 examples
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Convert text into numbers

ways to convert text to numbers:
1. Tokenization
2. Embedding


### Text vectorization

In [17]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [18]:
#Use the default textvectorization parameters
text_vectorizer = TextVectorization(max_tokens=None,#how many words in vocabulary(automatically add <OOV>)
                                    standardize="lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams=None,#create groups of n words
                                    output_mode ="int",#how to map tokens to numbers
                                    output_sequence_length=None)#,#how long do you want the sequences to be
                                    #pad_to_max_tokens=True)



In [19]:
#Find the average number of tokens in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [20]:
#setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15 #max length our sequence will be

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode ="int",
                                    output_sequence_length = max_length)

In [21]:
#Fit the text vectorizer to training sentence
text_vectorizer.adapt(train_sentences)

In [22]:
#createa a sample sentence and tokenize it
sample_sentence = "There's a flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [23]:
# Choose random sentence from training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text :\n {random_sentence}\
        \n\nVectorized version:")
text_vectorizer([random_sentence])

Original text :
 I will adamantly opposed to nuclear weapons.        

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  8,  38,   1,   1,   5, 105, 258,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [24]:
# get the unique word in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary() #get all the unique words in the vocabulary of training sentence
top_5_words = words_in_vocab[:5] #get most common word
bottom_5_words = words_in_vocab[-5:] #get least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most common words: {top_5_words}")
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


### Creating an Embedding using an embedding layer


for embedding we are gona use tensorflow embedding layer
parameters used:
1. input_dim - size of vocabulary
2. output_dim - size of the output embedding vector
3. input_length -length of sequences being passed to the embedding layer

In [25]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             embeddings_initializer="uniform",
                             input_length = max_length)
embedding

In [26]:
#get a random sentence from the training sentences
random_sentence = random.choice(train_sentences)
print(f"Original Text:\n {random_sentence}\
        \n\n Embedded version:")

#embed the random sentence
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original Text:
 SpaceX Founder Musk: Structural Failure Took Down Falcon 9 http://t.co/LvIzO9CSSR        

 Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-3.1123273e-03, -1.7416365e-03, -4.6144020e-02, ...,
          3.5506833e-02, -1.0830678e-02, -2.4720360e-02],
        [ 3.7033629e-02, -3.2475747e-02, -4.7893524e-02, ...,
         -3.0227650e-02,  1.5431885e-02,  2.6320815e-03],
        [ 3.7033629e-02, -3.2475747e-02, -4.7893524e-02, ...,
         -3.0227650e-02,  1.5431885e-02,  2.6320815e-03],
        ...,
        [-2.7273670e-03, -9.8001212e-05, -2.7609719e-02, ...,
         -4.1285641e-03, -2.2300852e-02,  9.3884356e-03],
        [-2.7273670e-03, -9.8001212e-05, -2.7609719e-02, ...,
         -4.1285641e-03, -2.2300852e-02,  9.3884356e-03],
        [-2.7273670e-03, -9.8001212e-05, -2.7609719e-02, ...,
         -4.1285641e-03, -2.2300852e-02,  9.3884356e-03]]], dtype=float32)>

In [27]:
#check out single tokens embedding
sample_embed[0][0],sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-3.11232731e-03, -1.74163654e-03, -4.61440198e-02,  1.17720738e-02,
         2.93276422e-02,  1.88767649e-02,  3.67266573e-02, -5.58320433e-03,
        -4.03473265e-02, -1.24875419e-02,  3.99736427e-02, -4.78344224e-02,
         1.47318505e-02, -4.85701337e-02,  1.17640011e-02,  2.85900347e-02,
        -1.85277686e-02,  4.74895351e-02, -9.96357203e-03,  3.60462107e-02,
         4.08053659e-02,  2.84940712e-02, -2.10400671e-03,  3.20814960e-02,
        -4.48545814e-02,  4.46104519e-02,  2.75981091e-02,  3.12420018e-02,
         1.43966712e-02,  4.11788858e-02, -4.19584028e-02,  3.32205184e-02,
        -8.39484856e-03,  3.09755094e-02, -2.22409889e-03,  2.31060423e-02,
        -3.91995423e-02, -2.41139531e-02, -4.73267697e-02, -4.60831188e-02,
        -2.61753798e-03,  3.92097943e-02, -2.59337667e-02,  2.16176249e-02,
        -6.28963858e-03,  3.51027586e-02, -1.82659142e-02, -1.69232003e-02,
         3.46398689e-02, -3.23648341e-0

## Modelling a text dataset(running a series of experiments)


series of model -start with baseline model

1. Model 0: Naive Bayes
2. Model 1: Feed forward neural network(dense model)
3. Model 2: LSTM model(RNN)
4. Model 3: GRU model(RNN)
5. Model 4: Bidirectional-LSTM model(RNN)
6. Model 5: 1D Convolutional Neural Network(CNN)
7. Model 6: TensorFlow Hub Pretrained Feature Extractor (using transfer learning for NLP)
8. Model 7: Same as model 6 with 10% of training data

Approach:
1. Create a model
2. Build a model
3. Fit a model
4. Evaluate model

### Model 0: Getting a baseline
 for baseline we will use sklearns multinomial Naive Bayes using TFIDF  formula to convert the words to numbers

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
#create tokenuization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf",MultinomialNB())
])

#Fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [29]:
#Evaluate the baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print((f"Baseline model accurcacy: {baseline_score*100:.2f}%"))

Baseline model accurcacy: 79.27%


In [30]:
#Make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating evaluation function for model experiments

Function to create model predictions with the true labels using the following metrics:
1. Accuracy
2. Precision
3. recall
4. F1-score


In [31]:
#Function to evaluate accuracy,precision,recall,f1-score
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
  #Calculate model ccuracy
  model_accuracy = accuracy_score(y_true,y_pred) *100
  #calculate model recall, precision and f1-scores using weighted average
  model_precision,model_recall,model_f1,_ = precision_recall_fscore_support(y_true,y_pred,average="weighted")
  model_results ={"accuracy": model_accuracy,
                  "precision": model_precision,
                  "recall": model_recall,
                  "f1": model_f1}
  return model_results

In [32]:
#get baseline results
baseline_results = calculate_results(y_true=val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

### Model 1: Simple Dense Model

In [33]:
#Create a tensorboard callback(need to create a new one for each)
from helper_functions import create_tensorboard_callback

#create a directory to save Tensorboard Logs
SAVE_DIR = "model_logs"

In [56]:
#Build model with FUnctional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string) #inputs are 1-dimensional strings
x = text_vectorizer(inputs) #turn input text to numbers
x = embedding(x) #create an embedding of the numberized inputs
x = layers.GlobalAveragePooling1D()(x) # lower the dimensionality of the embedding (try running the model without this layer and see what happens)
outputs = layers.Dense(1, activation="sigmoid")(x) # create output layer, want binary outputs so use sigmoid activation
model_1 = tf.keras.Model(inputs,outputs, name="model_1_dense")

In [41]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [42]:
#compile model
model_1.compile(loss="binary_crossentropy",
                optimizer= tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [43]:
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20230123-033344
Epoch 1/5
215/215 [==============================] - 4s 5ms/step - loss: 0.6107 - accuracy: 0.6894 - val_loss: 0.5383 - val_accuracy: 0.7507
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.4413 - accuracy: 0.8164 - val_loss: 0.4701 - val_accuracy: 0.7874
Epoch 3/5
215/215 [==============================] - 1s 4ms/step - loss: 0.3463 - accuracy: 0.8587 - val_loss: 0.4569 - val_accuracy: 0.7927
Epoch 4/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2836 - accuracy: 0.8918 - val_loss: 0.4641 - val_accuracy: 0.7874
Epoch 5/5
215/215 [==============================] - 1s 4ms/step - loss: 0.2370 - accuracy: 0.9126 - val_loss: 0.4829 - val_accuracy: 0.7822


In [44]:
#Check the results
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.4829 - accuracy: 0.7822


[0.48290932178497314, 0.7821522355079651]

In [46]:
from pandas.core.algorithms import mode
#make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

24/24 [==============================] - 0s 3ms/step


(762, 1)

In [47]:
model_1_pred_probs[0]

array([0.30801937], dtype=float32)

In [48]:
model_1_pred_probs[:10]

array([[0.30801937],
       [0.8249317 ],
       [0.99779105],
       [0.17951412],
       [0.10160463],
       [0.9577023 ],
       [0.9095152 ],
       [0.99320847],
       [0.97323024],
       [0.42337096]], dtype=float32)

In [49]:
#convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [50]:
#Calculate our model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'precision': 0.7846540517195822,
 'recall': 0.7821522309711286,
 'f1': 0.7799245444538409}

In [51]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [52]:
import numpy as np
np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

## Visualizing learned embedding

In [53]:
# get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [60]:
#Get the weight matrix of embedding layer
embed_weights = model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


Now we've got these two objects, we can use the Embedding Projector tool to visualize our embedding.

To use the Embedding Projector tool, we need two files:

1. The embedding vectors (same as embedding weights).

2. The meta data of the embedding vectors (the words they represent - our vocabulary).

In [61]:
#Create embedding files
# import io

# # # Create output writers
# out_v = io.open("vectors.tsv", "w", encoding="utf-8")
# out_m = io.open("metadata.tsv", "w", encoding="utf-8")

# # # Write embedding vectors and words to file
# for num, word in enumerate(words_in_vocab):
#   if num == 0: 
#      continue # skip padding token
#   vec = embed_weights[num]
#   out_m.write(word + "\n") # write words to file
#   out_v.write("\t".join([str(x) for x in vec]) + "\n") # write corresponding word vector to file
# out_v.close()
# out_m.close()

In [63]:
 # Download files locally to upload to Embedding Projector
# try:
#   from google.colab import files
# except ImportError:
#   pass
# else:
#   files.download("vectors.tsv")
#   files.download("metadata.tsv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN)

RNNs are useful for sequence Data

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input

### Model 2: LSTM

LTSM = long short term memory

structure of RNN typically looks like this:

Input(text) -> Tokenize -> EMbedding -> Layers (RNNS/dense) -> Output(label probability)

In [65]:
#Create an LTSM model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# print(x.shape)
# x = layers.LSTM(64,return_sequences=True)(x)
# print(x.shape)
x = layers.LSTM(64)(x)
# print(x.shape)
# x = layers.Dense(64, activation ="relu")(x)
outputs = layers.Dense(1, activation ="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs, name = "model_2_LSTM")

In [66]:
#summary
model_2.summary()

Model: "model_2_LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,329,473
Trainable params: 1,329,473
Non-trainable params: 0
____________________________________________

In [67]:
#COmpile the model
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [68]:
#Fit the model
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20230123-042955
Epoch 1/5
215/215 [==============================] - 6s 9ms/step - loss: 0.2254 - accuracy: 0.9142 - val_loss: 0.5957 - val_accuracy: 0.7769
Epoch 2/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1535 - accuracy: 0.9425 - val_loss: 0.5970 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 2s 7ms/step - loss: 0.1297 - accuracy: 0.9517 - val_loss: 0.6485 - val_accuracy: 0.7822
Epoch 4/5
215/215 [==============================] - 1s 7ms/step - loss: 0.1054 - accuracy: 0.9597 - val_loss: 0.7857 - val_accuracy: 0.7769
Epoch 5/5
215/215 [==============================] - 1s 7ms/step - loss: 0.0816 - accuracy: 0.9682 - val_loss: 0.9230 - val_accuracy: 0.7756


In [69]:
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

24/24 [==============================] - 1s 4ms/step


array([[1.7774412e-02],
       [8.8815951e-01],
       [9.9962676e-01],
       [1.6414300e-02],
       [2.3078029e-04],
       [9.9833328e-01],
       [6.7634374e-01],
       [9.9981767e-01],
       [9.9963605e-01],
       [3.5351828e-01]], dtype=float32)

In [70]:
#Convert model 2 pred probs to labels
model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [71]:
#Calculate model 2 results
model_2_results = calculate_results(y_true = val_labels,
                                    y_pred = model_2_preds)
model_2_results

{'accuracy': 77.55905511811024,
 'precision': 0.7769617885542762,
 'recall': 0.7755905511811023,
 'f1': 0.7737412623092624}

In [72]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

from tables.tests.common import unittest
### Model 3: GRU

ANother popular and effective RNN component is the GRU or gated recurrent unit 

The GRU cell has similar features to an LSTM cell but has less paramteres

In [79]:
#Build an RNN using GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype =tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)#, return_sequences=True)(x)
# x = layers.LSTM(42, return_sequences=True)(x)
# x = layers.GRU(99)(x)
#x = layers.GlobalAveragePooling1D()(x)
# x = layers.Dense(64, activation ="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_3 = tf.keras.Model(inputs,outputs, name="model_3_GRU")

In [80]:
model_3.summary()

Model: "model_3_GRU"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 gru_4 (GRU)                 (None, 64)                37248     
                                                                 
 dense_9 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_____________________________________________

In [81]:
#COmpile the model
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [82]:
#Fit the model
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20230123-044344
Epoch 1/5
215/215 [==============================] - 4s 9ms/step - loss: 0.1494 - accuracy: 0.9477 - val_loss: 0.7301 - val_accuracy: 0.7756
Epoch 2/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0816 - accuracy: 0.9689 - val_loss: 0.9095 - val_accuracy: 0.7769
Epoch 3/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0697 - accuracy: 0.9737 - val_loss: 0.8537 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0607 - accuracy: 0.9747 - val_loss: 1.0391 - val_accuracy: 0.7743
Epoch 5/5
215/215 [==============================] - 1s 6ms/step - loss: 0.0523 - accuracy: 0.9778 - val_loss: 1.1601 - val_accuracy: 0.7743


In [84]:
#Make predictions with GRU model
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

24/24 [==============================] - 0s 4ms/step


array([[2.0734516e-03],
       [7.6168448e-01],
       [9.9983883e-01],
       [1.2561238e-01],
       [1.1591028e-04],
       [9.9975687e-01],
       [5.9241712e-01],
       [9.9992383e-01],
       [9.9988246e-01],
       [9.7375363e-01]], dtype=float32)

In [85]:
#convert to labels
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [88]:
#Calculate model_3 results
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_preds)
model_3_results

{'accuracy': 77.42782152230971,
 'precision': 0.7759894665484696,
 'recall': 0.7742782152230971,
 'f1': 0.7722311836526509}

In [89]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

###Model 4: Bidirectional RNN

Normal RNNs go from left to right. However, birectional RNN goes from right to left as well as left to right

In [95]:
# Build a bidirectional RNN  in tensorflow
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [96]:
model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 bidirectional_4 (Bidirectio  (None, 128)              74496     
 nal)                                                            
                                                                 
 dense_12 (Dense)            (None, 1)                 129       
                                                                 
Total params: 1,354,625
Trainable params: 1,3

In [97]:
#Compile the model
model_4.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [98]:
#Fit the model
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_4_Birectional")])

Saving TensorBoard log files to: model_logs/model_4_Birectional/20230123-045533
Epoch 1/5
215/215 [==============================] - 6s 12ms/step - loss: 0.1224 - accuracy: 0.9599 - val_loss: 0.9519 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 2s 10ms/step - loss: 0.0574 - accuracy: 0.9769 - val_loss: 1.0985 - val_accuracy: 0.7638
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0486 - accuracy: 0.9781 - val_loss: 1.1906 - val_accuracy: 0.7651
Epoch 4/5
215/215 [==============================] - 2s 9ms/step - loss: 0.0452 - accuracy: 0.9781 - val_loss: 1.1496 - val_accuracy: 0.7690
Epoch 5/5
215/215 [==============================] - 2s 11ms/step - loss: 0.0427 - accuracy: 0.9800 - val_loss: 1.3257 - val_accuracy: 0.7782


In [100]:
#Make predictions with bidirectional model
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

24/24 [==============================] - 0s 5ms/step


array([[3.5870567e-04],
       [8.1241584e-01],
       [9.9979335e-01],
       [9.4822101e-02],
       [1.9307850e-05],
       [9.9890471e-01],
       [3.6957249e-01],
       [9.8715287e-01],
       [9.9987042e-01],
       [9.9736196e-01]], dtype=float32)

In [101]:
#Convert to labels
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))
model_4_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 0., 1., 1., 1.], dtype=float32)>

In [102]:
#Calculate model_4 results
model_4_results = calculate_results(y_true=val_labels,
                                    y_pred=model_4_preds)
model_4_results

{'accuracy': 77.82152230971128,
 'precision': 0.7833771178803304,
 'recall': 0.7782152230971129,
 'f1': 0.7748637513129987}

from tensorflow.python.ops.init_ops import ConvolutionOrthogonal1D
## Convolutional Neural Networks for Text(and other types of sequences)

We have used CNNS for images but images are typically 2D, however text data is model_1_pred_probs

Previously we have conv2D for our image data but now we are going to use ConvolutionOrthogonal1D

The typical structure of Conv1D model for sequences:

Inputs(Text) -> Tokenization -> Embedding -> Layers -> outputs

### Model 5: Conv1D

In [109]:
# Test out embedding layer, Conv1D layer and max pooling
from tensorflow.keras import layers
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters=64,
                       kernel_size=5,
                        strides=1,
                       activation="relu",
                       padding="same")#default - valid means the output is smaller than the input shape , same means output is same as input
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape,conv_1d_output.shape,max_pool_output.shape

(TensorShape([1, 15, 128]), TensorShape([1, 15, 64]), TensorShape([1, 64]))

In [113]:
#embedding_test

In [114]:
#conv_1d_output

In [115]:
#max_pool_output

In [120]:
# Create 1-d convolutional layer to model sequences
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=64,kernel_size=5,strides=1,activation="relu",padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

In [121]:
#compile the model
model_5.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [122]:
#model summary
model_5.summary()

Model: "model_5_Conv1D"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 conv1d_5 (Conv1D)           (None, 11, 64)            41024     
                                                                 
 global_max_pooling1d_5 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_13 (Dense)            (None, 1)              

In [123]:
#Fit the model
#Fit the model
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "model_5_Conv1D")])

Saving TensorBoard log files to: model_logs/model_5_Conv1D/20230123-052602
Epoch 1/5
215/215 [==============================] - 3s 6ms/step - loss: 0.1198 - accuracy: 0.9609 - val_loss: 0.9239 - val_accuracy: 0.7743
Epoch 2/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0723 - accuracy: 0.9731 - val_loss: 1.0455 - val_accuracy: 0.7743
Epoch 3/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0592 - accuracy: 0.9771 - val_loss: 1.1610 - val_accuracy: 0.7677
Epoch 4/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0573 - accuracy: 0.9759 - val_loss: 1.2166 - val_accuracy: 0.7638
Epoch 5/5
215/215 [==============================] - 1s 5ms/step - loss: 0.0507 - accuracy: 0.9791 - val_loss: 1.2255 - val_accuracy: 0.7625


In [124]:
#Make predictions with bidirectional model
model_5_pred_probs = model_5.predict(val_sentences)
model_5_pred_probs[:10]

24/24 [==============================] - 0s 2ms/step


array([[1.8706357e-01],
       [8.6516303e-01],
       [9.9991739e-01],
       [7.5676546e-02],
       [1.8838114e-07],
       [9.9593216e-01],
       [9.8416638e-01],
       [9.9997663e-01],
       [9.9999976e-01],
       [8.5212392e-01]], dtype=float32)

In [125]:
#Convert to labels
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))
model_5_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [126]:
#Calculate model_5 results
model_5_results = calculate_results(y_true=val_labels,
                                    y_pred=model_5_preds)
model_5_results

{'accuracy': 76.24671916010499,
 'precision': 0.7626600366229389,
 'recall': 0.7624671916010499,
 'f1': 0.7611077640788777}

### Model 6: Tensorflow Hub Pretrained Sentence Encoder

In [127]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples=embed([sample_sentence,
                      "When you call the universal sentence encoder on a sentence, it turns it into numbers."])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157028  0.0248591   0.02878048 -0.012715    0.03971538  0.0882776
  0.02680984  0.05589836 -0.0106873  -0.00597291  0.00639323 -0.01819518
  0.00030813  0.09105888  0.05874644 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865347 -0.04209306  0.02678981  0.03011006  0.00321069
 -0.00337973 -0.04787357  0.0226672  -0.00985925 -0.04063613 -0.01292092
 -0.04666384  0.05630299 -0.03949255  0.00517686  0.02495829 -0.0701444
  0.02871508  0.04947684 -0.00633979 -0.08960192  0.02807118 -0.00808364
 -0.01360602  0.0599865  -0.10361787 -0.05195374  0.00232954 -0.02332531
 -0.03758105  0.03327728], shape=(50,), dtype=float32)


In [128]:
embed_samples[0].shape

TensorShape([512])

In [129]:
# Create keras layer using the USE pretrained layer from tensorflow hub
sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[],
                                        dtype=tf.string,
                                        trainable=False,
                                        name="USE")

In [136]:
#Create model using Sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name="model_6_USE")

#Compile the model
model_6.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#summary
model_6.summary()

Model: "model_6_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_15 (Dense)            (None, 64)                32832     
                                                                 
 dense_16 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [137]:
#train a classifier on top of a USE pretrained model

model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder/20230123-054448
Epoch 1/5
215/215 [==============================] - 4s 14ms/step - loss: 0.5058 - accuracy: 0.7808 - val_loss: 0.4478 - val_accuracy: 0.7979
Epoch 2/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4164 - accuracy: 0.8137 - val_loss: 0.4387 - val_accuracy: 0.8058
Epoch 3/5
215/215 [==============================] - 3s 12ms/step - loss: 0.4003 - accuracy: 0.8222 - val_loss: 0.4356 - val_accuracy: 0.8097
Epoch 4/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3918 - accuracy: 0.8243 - val_loss: 0.4283 - val_accuracy: 0.8110
Epoch 5/5
215/215 [==============================] - 3s 12ms/step - loss: 0.3848 - accuracy: 0.8308 - val_loss: 0.4230 - val_accuracy: 0.8123


In [138]:
#Make predictions with bidirectional model
model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred_probs[:10]

24/24 [==============================] - 1s 10ms/step


array([[0.2024652 ],
       [0.8043865 ],
       [0.99076056],
       [0.22397119],
       [0.73219925],
       [0.8268656 ],
       [0.98339385],
       [0.9870724 ],
       [0.953359  ],
       [0.11034124]], dtype=float32)

In [139]:
#Convert to labels
model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
model_6_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [140]:
#Calculate model_6 results
model_6_results = calculate_results(y_true=val_labels,
                                    y_pred=model_6_preds)
model_6_results

{'accuracy': 81.23359580052494,
 'precision': 0.8131027998205999,
 'recall': 0.8123359580052494,
 'f1': 0.8113480882593861}

In [141]:
baseline_results

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

###Model 7: TF Hub Pretrained USE but with 10% of training data

Transfer learning helps when u dont have large dataset.
replicate model_6 but only on 10% data

In [145]:
#Create subset of 10% of the training data
train_10_percent = train_df_shuffled[["text","target"]].sample(frac=0.1,random_state=42)
train_sentences_10_percent = train_10_percent["text"].to_list()
train_labels_10_percent = train_10_percent["target"].to_list()
train_10_percent.shape,len(train_sentences_10_percent),len(train_labels_10_percent)

((761, 2), 761, 761)

In [143]:
#Check no of targets in subset data
train_10_percent["target"].value_counts()

0    413
1    348
Name: target, dtype: int64

To create a model the same as that of previous model, use tf.keras.models.clone_model method

In [149]:
#Create model using Sequential API
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name="model_7_USE")

# model_7 = tf.keras.models.clone_model(model_6)

#Compile the model
model_7.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#summary
model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 USE (KerasLayer)            (None, 512)               256797824 
                                                                 
 dense_17 (Dense)            (None, 64)                32832     
                                                                 
 dense_18 (Dense)            (None, 1)                 65        
                                                                 
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [150]:
#train a classifier on top of a USE pretrained model

model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                     "tf_hub_sentence_encoder_10_percent")])

Saving TensorBoard log files to: model_logs/tf_hub_sentence_encoder_10_percent/20230123-055411
Epoch 1/5
24/24 [==============================] - 3s 36ms/step - loss: 0.6585 - accuracy: 0.7227 - val_loss: 0.6112 - val_accuracy: 0.8031
Epoch 2/5
24/24 [==============================] - 1s 24ms/step - loss: 0.5747 - accuracy: 0.8095 - val_loss: 0.5293 - val_accuracy: 0.8110
Epoch 3/5
24/24 [==============================] - 1s 25ms/step - loss: 0.5022 - accuracy: 0.8003 - val_loss: 0.4660 - val_accuracy: 0.8045
Epoch 4/5
24/24 [==============================] - 1s 25ms/step - loss: 0.4516 - accuracy: 0.8081 - val_loss: 0.4270 - val_accuracy: 0.8228
Epoch 5/5
24/24 [==============================] - 0s 21ms/step - loss: 0.4194 - accuracy: 0.8213 - val_loss: 0.3995 - val_accuracy: 0.8360


In [151]:
#Make predictions with bidirectional model
model_7_pred_probs = model_7.predict(val_sentences)
model_7_pred_probs[:10]

24/24 [==============================] - 1s 17ms/step


array([[0.21061753],
       [0.8287078 ],
       [0.9161933 ],
       [0.30658865],
       [0.8176684 ],
       [0.81827307],
       [0.91825384],
       [0.9443213 ],
       [0.84772664],
       [0.05136238]], dtype=float32)

In [152]:
#Convert to labels
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 1., 1., 1., 1., 1., 0.], dtype=float32)>

In [153]:
#Calculate model_7 results
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred=model_7_preds)
model_7_results

{'accuracy': 83.59580052493439,
 'precision': 0.836332685135006,
 'recall': 0.8359580052493438,
 'f1': 0.835371693955307}